In [64]:
import pandas as pd
import numpy as np
import os
import re 

pd.set_option('max_columns', 50)

In [65]:
def load_statements_into_dict():

    df_dict = {}

    for dirname, _, filenames in os.walk(f"{csv_path}{ticker}"):
        if '(10-Q)' in dirname:
            for filename in filenames:
                #print(filename)
                if filename.lower() == '1002000 - Statement - Consolidated Statements Of Operations.csv'.lower():
                    full_path = os.path.join(dirname, filename)
                    df_dict[full_path[17:27]] = pd.read_csv(full_path,index_col=[0])
    return df_dict


def populate_time_series(df_dict,date_list):


    for date in date_list:
        
        date_cols = [date_col for date_col in df_dict[date].columns if re.match('[0-9]{4}-[0-9]{2}-[0-9]{2}',date_col)]
        tmp_df = df_dict[date][date_cols]

        if date == date_list[0]:
            master_df = tmp_df
        else:
            #df_dict.
            new_columns = tmp_df.columns.difference(master_df.columns)
            master_df = master_df.merge(tmp_df[new_columns],left_index=True,right_index=True,how='outer')
            master_df.loc[master_df.index.isin(tmp_df.index),master_df.columns.isin(tmp_df)] = tmp_df

    return master_df

def adjust_for_tag_changes(master_df,date_list): 
    for metric in master_df[(master_df.isna().any(axis=1) & master_df[date_list[0]].notna())].index:
        for metric_match in master_df[master_df.index != metric].index:

            non_nan_columns = master_df.loc[[metric,metric_match],:].notna().all(axis=0)
            agreement_series = master_df.loc[metric,non_nan_columns]==master_df.loc[metric_match,non_nan_columns]

            if (agreement_series.all() == True) and len(agreement_series > 1):
                    master_df.loc[metric,master_df.columns[master_df[master_df.index == metric].isna().iloc[0].to_list()]] = master_df.loc[metric_match,master_df.columns[master_df[master_df.index == metric].isna().iloc[0].to_list()]] 
                    master_df = master_df.drop(metric_match,axis=0)

    return master_df

#def adjust_by_matching_labels(master_df,df_dict):




def reorder_dataframe(master_df):
    latest_filing_mask = master_df.index.isin(df_dict[date_list[0]].index)

    master_df_tmp = master_df.loc[latest_filing_mask,:]
    master_df_tmp = master_df_tmp.reindex(df_dict[date_list[0]].index)
    master_df_tmp = master_df_tmp.append(master_df.loc[~latest_filing_mask,:])

    return master_df_tmp



In [67]:
csv_path = '../data/csv/'
ticker = 'AMZN'

df_dict = load_statements_into_dict()
date_list = sorted(list(df_dict.keys()),reverse=True)
master_df = populate_time_series(df_dict,date_list)
master_df = adjust_for_tag_changes(master_df,date_list)
master_df = reorder_dataframe(master_df)

master_df

,2020-03-31,2019-03-31,2018-09-30,2019-09-30,2018-06-30,2019-06-30,2018-03-31,2017-09-30,2017-06-30,2017-03-31,2016-09-30,2016-06-30,2016-03-31,2015-09-30,2015-06-30,2015-03-31,2014-09-30,2014-06-30,2014-03-31,2013-09-30,2013-06-30,2013-03-31,2012-09-30
consolidatedstatementsofoperations,,,,,,,,,,,,,,,,,,,,,,,
us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,7.545200e+10,5.970000e+10,5.657600e+10,6.998100e+10,5.288600e+10,6.340400e+10,5.104200e+10,4.374400e+10,3.795500e+10,3.571400e+10,3.271400e+10,3.040400e+10,2.912800e+10,2.535800e+10,2.318500e+10,2.271700e+10,2.057900e+10,1.934000e+10,1.974100e+10,1.709200e+10,1.570400e+10,1.607000e+10,1.380600e+10
us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,4.184100e+10,3.428300e+10,3.374600e+10,3.972600e+10,3.186400e+10,3.585600e+10,3.160500e+10,2.876800e+10,2.474500e+10,2.373400e+10,2.233900e+10,2.111600e+10,2.058100e+10,1.846300e+10,1.710400e+10,1.708400e+10,1.602200e+10,1.525100e+10,1.570500e+10,1.380800e+10,1.275200e+10,1.327100e+10,1.154600e+10
us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,3.361100e+10,2.541700e+10,2.283000e+10,3.025500e+10,2.102200e+10,2.754800e+10,1.943700e+10,1.497600e+10,1.321000e+10,1.198000e+10,1.037500e+10,9.288000e+09,8.547000e+09,6.895000e+09,6.081000e+09,5.633000e+09,4.557000e+09,4.089000e+09,4.036000e+09,3.284000e+09,2.952000e+09,2.799000e+09,2.260000e+09
us-gaap_costofgoodsandservicessold,4.425700e+10,3.392000e+10,3.300300e+10,4.130200e+10,3.063200e+10,3.633700e+10,3.073500e+10,2.754900e+10,2.345100e+10,2.244000e+10,2.126000e+10,1.918000e+10,1.886600e+10,1.675500e+10,1.516000e+10,1.539500e+10,1.462700e+10,1.339900e+10,1.405500e+10,1.236600e+10,1.120900e+10,1.180100e+10,1.031900e+10
amzn_fulfillmentexpense,1.153100e+10,8.601000e+09,8.275000e+09,1.016700e+10,7.932000e+09,9.271000e+09,7.792000e+09,6.420000e+09,5.158000e+09,4.697000e+09,4.335000e+09,3.878000e+09,3.687000e+09,3.230000e+09,2.876000e+09,2.759000e+09,2.643000e+09,2.382000e+09,2.317000e+09,2.034000e+09,1.837000e+09,1.796000e+09,1.510000e+09
us-gaap_marketingexpense,4.828000e+09,3.664000e+09,3.303000e+09,4.752000e+09,2.901000e+09,4.291000e+09,2.699000e+09,2.479000e+09,2.229000e+09,1.920000e+09,1.738000e+09,1.546000e+09,1.436000e+09,1.264000e+09,1.150000e+09,1.083000e+09,9.930000e+08,9.430000e+08,8.700000e+08,6.940000e+08,6.750000e+08,6.320000e+08,5.400000e+08
amzn_technologyandcontentexpense,9.325000e+09,7.927000e+09,7.162000e+09,9.200000e+09,7.247000e+09,9.065000e+09,6.759000e+09,5.944000e+09,5.549000e+09,4.813000e+09,4.135000e+09,3.880000e+09,3.526000e+09,3.197000e+09,3.020000e+09,2.754000e+09,2.423000e+09,2.226000e+09,1.991000e+09,1.734000e+09,1.586000e+09,1.383000e+09,1.192000e+09
us-gaap_generalandadministrativeexpense,1.452000e+09,1.173000e+09,1.041000e+09,1.348000e+09,1.111000e+09,1.270000e+09,1.067000e+09,9.600000e+08,8.740000e+08,7.950000e+08,6.390000e+08,5.800000e+08,4.970000e+08,4.630000e+08,4.670000e+08,4.270000e+08,4.060000e+08,3.770000e+08,3.270000e+08,2.780000e+08,2.860000e+08,2.460000e+08,2.300000e+08
us-gaap_otheroperatingincomeexpensenet,-7.000000e+07,5.000000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
for tag in master_df.loc[master_df.isna().any(axis=1),:].index:
    print(tag)
    na_ded = master_df.loc[tag,master_df.loc[tag,].notna()].index.max()
    label_roles = ['label','terseLab','verboseLabel']
    for label in label_roles:
        if label in df_dict[na_ded].columns:
            print(df_dict[na_ded].loc[tag,label])

us-gaap_otheroperatingincomeexpensenet
Other Operating Income (Expense), Net
nan
us-gaap_othercostandexpenseoperating
Other Cost and Expense, Operating
nan
